In [1]:
# Bayes Factor function from the originnal TADA model. 
# In this function, the original code uses the "dnbinnom" function of R.

In [2]:
# Bayes factor of de novo counts of a gene 
# x: the de novo count
# N: the sample size (number of families)
# mu: the mutation rate (of this type of mutational events)
# Prior distribution of RR: gamma ~ Gamma(gamma.mean*beta, beta)
bayes.factor.denovo <- function(x, N, mu, gamma.mean, beta) {
  marg.lik0 <- dpois(x, 2*N*mu)
  marg.lik1 <- dnbinom(x, gamma.mean*beta, beta/(beta+2*N*mu))
  BF <- marg.lik1/marg.lik0
  
  return (BF)
}


In [3]:
# Bayes Factor function for numerical integration. 
# We DO NOT USE the built-in function "dnbinom" of R, and set "subdivisions = 500L" here

In [4]:
##################Use numerical integration for marg.lik1
##Note: the parameter "subdivisionns" can affect final results.

bayes.factor.denovo1 <- function(x, N, mu, gamma.mean, beta) {

    ##Model 0
    marg.lik0 <- dpois(x, 2*N*mu)
    ##Model 1
    ###Create a function for x ~ Poisson(2*N*mu*x.gamma) and x.gamma ~ Gamma(gamma.mean*beta, beta)
    fM1 <- function(x.gamma){
      dpois(x, lambda = 2*N*mu*x.gamma)*dgamma(x.gamma, shape = gamma.mean*beta, rate = beta)
    }
    ###Generate the lower and upper gamma values
    x.gamma.temp <- rgamma(1000, shape = gamma.mean*beta, rate = beta)
    gammaMin <- min(x.gamma.temp)
    gammaMax <- max(x.gamma.temp)
    
    marg.lik1 <- integrate(fM1, gammaMin, gammaMax, subdivisions = 500L)$value

    BF <- marg.lik1/marg.lik0
  
  return (BF)
}


In [5]:
# We will run some examples to compare results 

In [6]:
###Some examples
N = 10000
beta = 1
mu = 0.000018
gamma.mean = 20
x = 2

outData <- NULL
for (mu in c(10^-6, 10^-8, 10^-10)){
    for (x in 0:3){
        bf.from.TADA <- bayes.factor.denovo(x = x, N = N, mu = mu, 
                                            gamma.mean = gamma.mean, 
                                            beta = beta)

        bf.from.numericalInt <- bayes.factor.denovo1(x = x, N = N, mu = mu, 
                                                     gamma.mean = gamma.mean, 
                                                     beta = beta)

        outData <- rbind(outData,
                         c(mu, x, bf.from.TADA, bf.from.numericalInt))
        
    }}


colnames(outData) <- c("MutationRate", "DenovoMutationNumber", 
                       "BayesFactorFromTADA", "BayesFactorFromNumericalIntegration")
outData


MutationRate,DenovoMutationNumber,BayesFactorFromTADA,BayesFactorFromNumericalIntegration
1e-06,0,0.6865663,0.6862166
1e-06,1,13.4620834,13.4095888
1e-06,2,277.1605416,276.7072410
1e-06,3,5977.9724653,5896.0210919
1e-08,0,0.9962076,0.9951913
1e-08,1,19.9201682,19.9109932
1e-08,2,418.2398832,418.1170221
1e-08,3,9199.4375433,9141.5826462
1e-10,0,0.9999620,0.9977574
1e-10,1,19.9992000,19.9742497
